In [2]:
from langchain.llms import OpenAI

In [4]:
import os
os.environ['OPEN_API_KEY']=''

In [5]:
llm=OpenAI(model='gpt-3.5-turbo-0125',openai_api_key=os.environ['OPEN_API_KEY'],temperature=0.6)
#temperature closer to 0, means the model is safe and not taking any bets
#temperature closer to 1, means the model might generate wrong outputs but will be more creative

c:\Users\satya\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
text="What is the capital of India"
llm.invoke(text)

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [13]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=''

In [25]:
from langchain_community.llms import HuggingFaceHub

llm_huggingface = HuggingFaceEndpoint(repo_id='google/flan-t5-large', temperature=0, max_new_tokens=250, max_length=64)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kyouma45/.cache/huggingface/token
Login successful


In [26]:
output=llm_huggingface.predict('What is the capital of Russia?')
print(output)

BadRequestError:  (Request ID: ipPnWQhuaiYmrmmIQ8kOD)

Bad request:
The following `model_kwargs` are not used by the model: ['return_full_text', 'stop', 'watermark', 'stop_sequences'] (note: typos in the generate arguments will also show up in this list)

# Prompt Templates And LLMChain

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="India")

In [ ]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("India"))

# Simple Sequential Chain

In [ ]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

In [ ]:
famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

# Sequential Chain

In [ ]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [ ]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [ ]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [ ]:
chain({'country':"India"})

# Chatmodels With ChatOpenAI

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [ ]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [ ]:
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

# Prompt Template + LLM +Output Parsers

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [ ]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [ ]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})